In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
from datetime import datetime
import warnings
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import json
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False
tf.random.set_seed(13)

In [2]:
global_epoch_number = 100

## Reading data

In [3]:
# reading data
evo_data = pd.read_csv('data/interpol/evo_interpol_demand.csv', index_col=0)
modo_data = pd.read_csv('data/interpol/modo_interpol_demand.csv', index_col=0)
c2g_data = pd.read_csv('data/interpol/c2g_interpol_demand.csv', index_col=0)

In [4]:
evo_data.columns

Index(['tempC', 'precipMM', 'FeelsLikeC', 'uvIndex', 'visibility',
       'windspeedMiles', 'Blizzard', 'Clear', 'Cloudy', 'Fog', 'Heavy rain',
       'Heavy rain at times', 'Heavy snow', 'Light drizzle', 'Light rain',
       'Light rain shower', 'Light sleet', 'Light sleet showers', 'Light snow',
       'Mist', 'Moderate or heavy freezing rain',
       'Moderate or heavy rain shower', 'Moderate or heavy rain with thunder',
       'Moderate or heavy sleet', 'Moderate or heavy snow showers',
       'Moderate or heavy snow with thunder', 'Moderate rain',
       'Moderate rain at times', 'Moderate snow', 'Overcast', 'Partly cloudy',
       'Patchy heavy snow', 'Patchy light drizzle', 'Patchy light rain',
       'Patchy light rain with thunder', 'Patchy light snow',
       'Patchy moderate snow', 'Patchy rain possible', 'Patchy sleet possible',
       'Patchy snow possible', 'Sunny', 'Thundery outbreaks possible',
       'Torrential rain shower', 'Monday', 'Tuesday', 'Wednesday', 'Thursday

In [5]:
evo_data.drop(columns = ['hour_0', 'hour_1', 'hour_2', 'hour_3',
       'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10',
       'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16',
       'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22',
       'hour_23'], inplace=True)
modo_data.drop(columns = ['hour_0', 'hour_1', 'hour_2', 'hour_3',
       'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10',
       'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16',
       'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22',
       'hour_23'], inplace=True)
c2g_data.drop(columns = ['hour_0', 'hour_1', 'hour_2', 'hour_3',
       'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10',
       'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16',
       'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22',
       'hour_23'], inplace=True)

In [6]:
unievo_data = pd.DataFrame(evo_data.travels)
unimodo_data = pd.DataFrame(modo_data.travels)
unic2g_data = pd.DataFrame(c2g_data.travels)

In [7]:
init_period = '06-23'
end_period = '07-12'

evo_data = evo_data[(evo_data.index >= '2018-'+init_period) & (evo_data.index <= '2018-'+end_period)]
modo_data = modo_data[(modo_data.index >= '2018-'+init_period) & (modo_data.index <= '2018-'+end_period)]
c2g_data = c2g_data[(c2g_data.index >= '2017-'+init_period) & (c2g_data.index <= '2017-'+end_period)]

unievo_data = unievo_data[(unievo_data.index >= '2018-'+init_period) & (unievo_data.index <= '2018-'+end_period)]
unimodo_data = unimodo_data[(unimodo_data.index >= '2018-'+init_period) & (unimodo_data.index <= '2018-'+end_period)]
unic2g_data = unic2g_data[(unic2g_data.index >= '2017-'+init_period) & (unic2g_data.index <= '2017-'+end_period)]

In [8]:
# Adding the Canada Day Holiday
evo_data.index = pd.to_datetime(evo_data.index)
modo_data.index = pd.to_datetime(modo_data.index)
c2g_data.index = pd.to_datetime(c2g_data.index)

evo_data["holidays"] = pd.Series()
modo_data["holidays"] = pd.Series()
c2g_data["holidays"] = pd.Series()

evo_data["holidays"] = evo_data["holidays"].fillna(0)
modo_data["holidays"] = modo_data["holidays"].fillna(0)
c2g_data["holidays"] = c2g_data["holidays"].fillna(0)

canada_day = datetime(2018, 7, 1)
end_canada_day = datetime(2018,7 ,3)

evo_data.loc[((evo_data.index > canada_day) & (evo_data.index <= end_canada_day))]["holidays"] = 1
modo_data.loc[((modo_data.index > canada_day) & (modo_data.index <= end_canada_day))]["holidays"] = 1
c2g_data.loc[((c2g_data.index > canada_day) & (c2g_data.index <= end_canada_day))]["holidays"] = 1

## LSTM Data Preparation

In [9]:
def sup_learning_formatter(data, past_lags, future_steps, train_split):
    X = []
    y = []

    norm_data  = data.values

    for n in range(len(data) - past_lags - future_steps):
        X.append(norm_data[n : n + past_lags])
        y.append(data.travels.values[n + past_lags : n + past_lags + future_steps])
    return np.array(X), np.array(y)

In [10]:
def train_val_test_splitter(data, splits):
    locs = [int(len(data)*n) for n in splits]
    return data[:locs[0]], data[locs[0]:locs[1]], data[locs[1]:], data[0].shape

In [11]:
def eval_model(y, y_hat):
    evaluation = {}
    evaluation["RMSE"] = np.sqrt(mean_squared_error(y, y_hat))
    evaluation["MAE"] = mean_absolute_error(y, y_hat)
    evaluation["R2"] = r2_score(y, y_hat)

    return evaluation

In [12]:
def persistance_model(X, timesteps):
    y_hat = []
    for x in X:
        y_hat.append(np.array([x[-1][0] for _ in range(timesteps)]))

    return np.array(y_hat)

# Training Models

In [13]:
def plot_train_history(history, title, save_file=False):
    history = pd.DataFrame(history.history)

    history.plot(figsize=(8, 5))
    plt.grid(True)
    plt.savefig("plots\\" + title.replace(" ", "_") + ".png", bbox_inches='tight') if save_file else print()
    plt.show()

Next, will be generated the model for each dataset

## Grid Search

In [14]:
class GridSearchLSTM:
    def __init__(self):
        self.evaluations = {}
        self.best_estimator = None

    def search(self, feature_dict, data, verbose=1, windows=1, splits = (0.6, 0.8)):
        
        def average_evaluations(validation_eval, key="val_loss"):
            acc_value = 0
            for split, evaluation in validation_eval:
                acc_value += evaluation[key]
            return acc_value/len(validation_eval)
        
        possibilities_list = self._create_feature_dict(feature_dict)
        current_evaluations = []
        if(windows == 1):
            for test in tqdm(possibilities_list):
                model, hist, test_data, evaluation = self.run_lstm(data, 24, 12, splits, verbose=verbose, **test)
                validation_eval = {key:value[-1] for key, value in hist.history.items()}
                current_evaluations.append([test, validation_eval])
        
        else:
            increase = splits[1]/(windows + 1)
            for test in tqdm(possibilities_list):
                validation_eval = []
                for i in tqdm(range(windows)):
                    cur_split = (increase*(i + 1), increase*(i + 2))
                    model, hist, test_data, evaluation = self.run_lstm(data, 24, 12, cur_split, verbose=verbose, **test)
                    cur_validation_eval = {key:value[-1] for key, value in hist.history.items()}
                    validation_eval.append([cur_split, cur_validation_eval])
                    print(cur_split)
                current_evaluations.append([test, validation_eval])
        
        if(windows == 1):
            current_evaluations.sort(key=lambda x: x[1]["val_loss"])
        else:
            current_evaluations.sort(key=lambda x: average_evaluations(x[1]))   
        
        self.evaluations = current_evaluations
        self.best_estimator = current_evaluations[0][0]
            

    def _create_feature_dict(self, feature_dict):
        return self._create_feature_dict_recurse({}, feature_dict, list(feature_dict.keys()))

    def _create_feature_dict_recurse(self, start_dict, feature_dict, remaining_keys):
        if len(remaining_keys) == 0:
            return [start_dict]
        new_feature_dict = feature_dict.copy()
        returned_list = []
        del new_feature_dict[remaining_keys[0]]
        for item in feature_dict[remaining_keys[0]]:
            new_start_dict = start_dict.copy()
            new_start_dict[remaining_keys[0]] = item
            returned_list += self._create_feature_dict_recurse(new_start_dict, new_feature_dict, remaining_keys[1:])
        return returned_list


    def run_lstm(self, data, past_lags, future_steps, splits, node_number=50,
                 epochs=10, batch_size=10000, loss='mae', dropout=0.5, layer_count=2, verbose=1):
        
        X, y = sup_learning_formatter(data, past_lags, future_steps, splits[0])
        X_train, X_val, X_test, X_shape = train_val_test_splitter(X, splits)
        y_train, y_val, y_test, y_shape = train_val_test_splitter(y, splits)


        train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
        train = train.cache().shuffle(batch_size).batch(batch_size).repeat()

        val = tf.data.Dataset.from_tensor_slices((X_val, y_val))
        val = val.batch(batch_size).repeat()

        model = tf.keras.models.Sequential()

        if(layer_count == 1):
            model.add(tf.keras.layers.LSTM(node_number,
                                    input_shape=X_shape))
            model.add(tf.keras.layers.Dropout(dropout))
        else:
            model.add(tf.keras.layers.LSTM(node_number, return_sequences=True,
                                    input_shape=X_shape))
            model.add(tf.keras.layers.Dropout(dropout))

            for _ in range(layer_count - 2):
                model.add(tf.keras.layers.LSTM(node_number, return_sequences=True, activation='relu'))

            model.add(tf.keras.layers.LSTM(node_number, activation='relu'))
        
        model.add(tf.keras.layers.Dense(12))
        
        def rmse(y_true, y_pred):
            return tf.sqrt(tf.reduce_mean((y_true - y_pred)**2))

        model.compile(optimizer=tf.keras.optimizers.RMSprop(clipvalue=1.0), loss=loss, metrics=[rmse])
        
        history = model.fit(train, epochs=epochs, steps_per_epoch=50,
                            validation_data=val, validation_steps=50, verbose=verbose
                            )
        y_hat_test = model.predict(X_test)
        evaluation = eval_model(y_test, y_hat_test)

        return model, history, (X_test, y_test), evaluation


In [15]:
grid_search = GridSearchLSTM()

## Test Windows

In [16]:
feature_dict = {"epochs":[25], "layer_count":[5, 6, 7], "node_number":[60, 80, 110], "dropout":[0.8]}

grid_search.search(feature_dict, unimodo_data, windows=3)

Train for 50 steps, validate for 50 steps
Epoch 1/25
50/50 [==============================] - 12s 245ms/step - loss: 28.5294 - rmse: 36.4201 - val_loss: 31.7608 - val_rmse: 40.5342
Epoch 2/25
50/50 [==============================] - 5s 103ms/step - loss: 16.7143 - rmse: 21.4995 - val_loss: 24.5648 - val_rmse: 32.0105
Epoch 3/25
50/50 [==============================] - 5s 110ms/step - loss: 12.6173 - rmse: 16.9347 - val_loss: 22.0145 - val_rmse: 29.0612
Epoch 4/25
50/50 [==============================] - 5s 107ms/step - loss: 10.6354 - rmse: 14.5317 - val_loss: 26.3970 - val_rmse: 34.0070
Epoch 5/25
50/50 [==============================] - 5s 109ms/step - loss: 9.4263 - rmse: 13.0071 - val_loss: 22.8315 - val_rmse: 30.0719
Epoch 6/25
50/50 [==============================] - 5s 104ms/step - loss: 8.6712 - rmse: 11.9905 - val_loss: 16.0557 - val_rmse: 22.1583
Epoch 7/25
50/50 [==============================] - 5s 103ms/step - loss: 8.0341 - rmse: 11.1814 - val_loss: 17.7622 - val_rmse: 24

Train for 50 steps, validate for 50 steps
Epoch 1/25
50/50 [==============================] - 14s 280ms/step - loss: 25.5750 - rmse: 31.8826 - val_loss: 25.8322 - val_rmse: 33.6345
Epoch 2/25
50/50 [==============================] - 7s 144ms/step - loss: 17.8746 - rmse: 23.0256 - val_loss: 23.1886 - val_rmse: 30.2899
Epoch 3/25
50/50 [==============================] - 7s 141ms/step - loss: 13.7555 - rmse: 18.1278 - val_loss: 25.5687 - val_rmse: 32.9321
Epoch 4/25
50/50 [==============================] - 7s 145ms/step - loss: 11.2072 - rmse: 15.1408 - val_loss: 22.4751 - val_rmse: 29.4719
Epoch 5/25
50/50 [==============================] - 7s 142ms/step - loss: 9.9087 - rmse: 13.5621 - val_loss: 14.4107 - val_rmse: 20.1175
Epoch 6/25
50/50 [==============================] - 7s 147ms/step - loss: 9.0476 - rmse: 12.4583 - val_loss: 19.2112 - val_rmse: 25.9733
Epoch 7/25
50/50 [==============================] - 7s 143ms/step - loss: 8.5586 - rmse: 11.8298 - val_loss: 16.9062 - val_rmse: 23

Train for 50 steps, validate for 50 steps
Epoch 1/25
50/50 [==============================] - 22s 435ms/step - loss: 102.8393 - rmse: 135.9978 - val_loss: 25.0203 - val_rmse: 31.8357
Epoch 2/25
50/50 [==============================] - 15s 304ms/step - loss: 19.5598 - rmse: 24.6293 - val_loss: 26.5922 - val_rmse: 34.9624
Epoch 3/25
50/50 [==============================] - 15s 303ms/step - loss: 18.0679 - rmse: 23.2772 - val_loss: 31.1781 - val_rmse: 39.8506
Epoch 4/25
50/50 [==============================] - 15s 304ms/step - loss: 16.5830 - rmse: 21.5180 - val_loss: 24.3348 - val_rmse: 32.1353
Epoch 5/25
50/50 [==============================] - 15s 305ms/step - loss: 13.6556 - rmse: 18.0701 - val_loss: 10.5253 - val_rmse: 14.8080
Epoch 6/25
50/50 [==============================] - 15s 305ms/step - loss: 11.5053 - rmse: 15.5169 - val_loss: 13.5663 - val_rmse: 19.3995
Epoch 7/25
50/50 [==============================] - 15s 302ms/step - loss: 10.0413 - rmse: 13.7175 - val_loss: 12.1791 - v

Train for 50 steps, validate for 50 steps
Epoch 1/25
50/50 [==============================] - 17s 333ms/step - loss: 23.4449 - rmse: 29.2037 - val_loss: 25.3215 - val_rmse: 33.6867
Epoch 2/25
50/50 [==============================] - 9s 185ms/step - loss: 17.9408 - rmse: 22.8572 - val_loss: 19.2490 - val_rmse: 26.5963
Epoch 3/25
50/50 [==============================] - 9s 186ms/step - loss: 13.8187 - rmse: 18.3209 - val_loss: 15.6005 - val_rmse: 22.8217
Epoch 4/25
50/50 [==============================] - 9s 187ms/step - loss: 11.4675 - rmse: 15.5538 - val_loss: 19.0582 - val_rmse: 26.8704
Epoch 5/25
50/50 [==============================] - 10s 190ms/step - loss: 10.2320 - rmse: 13.9915 - val_loss: 11.8008 - val_rmse: 16.8824
Epoch 6/25
50/50 [==============================] - 9s 187ms/step - loss: 9.2417 - rmse: 12.7345 - val_loss: 20.3423 - val_rmse: 28.0441
Epoch 7/25
50/50 [==============================] - 9s 185ms/step - loss: 8.6852 - rmse: 12.0060 - val_loss: 21.1703 - val_rmse: 

Train for 50 steps, validate for 50 steps
Epoch 1/25
50/50 [==============================] - 20s 397ms/step - loss: 25.0628 - rmse: 32.1952 - val_loss: 28.7390 - val_rmse: 37.2254
Epoch 2/25
50/50 [==============================] - 12s 241ms/step - loss: 18.9362 - rmse: 24.7381 - val_loss: 22.6778 - val_rmse: 30.1309
Epoch 3/25
50/50 [==============================] - 12s 245ms/step - loss: 15.9317 - rmse: 20.9526 - val_loss: 25.2385 - val_rmse: 32.5623
Epoch 4/25
50/50 [==============================] - 12s 244ms/step - loss: 12.1942 - rmse: 16.3930 - val_loss: 21.6644 - val_rmse: 28.5047
Epoch 5/25
50/50 [==============================] - 12s 243ms/step - loss: 10.6468 - rmse: 14.5183 - val_loss: 18.3337 - val_rmse: 24.9088
Epoch 6/25
50/50 [==============================] - 12s 242ms/step - loss: 9.7097 - rmse: 13.2656 - val_loss: 21.0927 - val_rmse: 27.9924
Epoch 7/25
50/50 [==============================] - 12s 246ms/step - loss: 9.0611 - rmse: 12.4557 - val_loss: 14.2043 - val_r

Train for 50 steps, validate for 50 steps
Epoch 1/25
50/50 [==============================] - 29s 575ms/step - loss: 208.1158 - rmse: 282.4236 - val_loss: 25.2305 - val_rmse: 32.5140
Epoch 2/25
50/50 [==============================] - 22s 438ms/step - loss: 34.9282 - rmse: 45.5412 - val_loss: 53.1427 - val_rmse: 68.2689
Epoch 3/25
50/50 [==============================] - 22s 436ms/step - loss: 23.2862 - rmse: 29.4790 - val_loss: 26.7357 - val_rmse: 34.1582
Epoch 4/25
50/50 [==============================] - 22s 438ms/step - loss: 24.2747 - rmse: 32.5477 - val_loss: 25.6762 - val_rmse: 32.9703
Epoch 5/25
50/50 [==============================] - 22s 434ms/step - loss: 24.5439 - rmse: 31.1382 - val_loss: 23.0422 - val_rmse: 28.5075
Epoch 6/25
50/50 [==============================] - 22s 435ms/step - loss: 19.4567 - rmse: 24.2105 - val_loss: 29.8934 - val_rmse: 38.8844
Epoch 7/25
50/50 [==============================] - 22s 438ms/step - loss: 14.0337 - rmse: 18.3656 - val_loss: 15.8650 - v

Train for 50 steps, validate for 50 steps
Epoch 1/25
50/50 [==============================] - 20s 400ms/step - loss: 25.1893 - rmse: 32.0602 - val_loss: 33.9657 - val_rmse: 42.9114
Epoch 2/25
50/50 [==============================] - 11s 224ms/step - loss: 18.5169 - rmse: 23.8289 - val_loss: 23.3715 - val_rmse: 31.2964
Epoch 3/25
50/50 [==============================] - 11s 224ms/step - loss: 15.1113 - rmse: 19.8214 - val_loss: 26.7939 - val_rmse: 34.6556
Epoch 4/25
50/50 [==============================] - 11s 222ms/step - loss: 12.6782 - rmse: 17.0613 - val_loss: 26.0720 - val_rmse: 33.9613
Epoch 5/25
50/50 [==============================] - 11s 223ms/step - loss: 11.5888 - rmse: 15.9431 - val_loss: 24.8203 - val_rmse: 32.2461
Epoch 6/25
50/50 [==============================] - 11s 223ms/step - loss: 10.7162 - rmse: 14.7548 - val_loss: 15.8986 - val_rmse: 22.5175
Epoch 7/25
50/50 [==============================] - 11s 227ms/step - loss: 9.6555 - rmse: 13.2691 - val_loss: 15.5902 - val_

Train for 50 steps, validate for 50 steps
Epoch 1/25
50/50 [==============================] - 27s 543ms/step - loss: 175.2437 - rmse: 242.2475 - val_loss: 29.5760 - val_rmse: 37.0691
Epoch 2/25
50/50 [==============================] - 19s 372ms/step - loss: 50.7763 - rmse: 67.1201 - val_loss: 25.5924 - val_rmse: 33.7042
Epoch 3/25
50/50 [==============================] - 19s 373ms/step - loss: 23.0665 - rmse: 28.8698 - val_loss: 24.7997 - val_rmse: 33.1255
Epoch 4/25
50/50 [==============================] - 18s 366ms/step - loss: 24.0506 - rmse: 30.8483 - val_loss: 26.6766 - val_rmse: 33.6381
Epoch 5/25
50/50 [==============================] - 19s 375ms/step - loss: 24.5914 - rmse: 32.0639 - val_loss: 27.3682 - val_rmse: 35.7743
Epoch 6/25
50/50 [==============================] - 19s 378ms/step - loss: 19.9484 - rmse: 25.9400 - val_loss: 24.1907 - val_rmse: 29.5885
Epoch 7/25
50/50 [==============================] - 18s 367ms/step - loss: 20.3257 - rmse: 25.5010 - val_loss: 26.2280 - v

Train for 50 steps, validate for 50 steps
Epoch 1/25
50/50 [==============================] - 44s 887ms/step - loss: 413.7067 - rmse: 560.1672 - val_loss: 159.4156 - val_rmse: 205.7175
Epoch 2/25
50/50 [==============================] - 36s 720ms/step - loss: 188.4518 - rmse: 257.8101 - val_loss: 28.8717 - val_rmse: 36.6497
Epoch 3/25
50/50 [==============================] - 36s 726ms/step - loss: 100.3073 - rmse: 133.7411 - val_loss: 365.3354 - val_rmse: 489.0358
Epoch 4/25
50/50 [==============================] - 36s 725ms/step - loss: 135.9535 - rmse: 187.0481 - val_loss: 170.1585 - val_rmse: 224.5976
Epoch 5/25
50/50 [==============================] - 36s 728ms/step - loss: 518.9847 - rmse: 712.7269 - val_loss: 95.0337 - val_rmse: 120.1593
Epoch 6/25
50/50 [==============================] - 36s 729ms/step - loss: 209.5094 - rmse: 290.9216 - val_loss: 54.1047 - val_rmse: 71.4482
Epoch 7/25
50/50 [==============================] - 36s 728ms/step - loss: 211.9690 - rmse: 307.7745 - va

In [17]:
def evaluator(x):
    return sum([splits[-1]*values['val_loss'] for (splits, values) in x])

In [18]:
grid_search.evaluations.sort(key=lambda x:evaluator(x[1]))
grid_search.evaluations #Unimodo

[[{'epochs': 25, 'layer_count': 7, 'node_number': 110, 'dropout': 0.8},
  [[(0.2, 0.4),
    {'loss': 6.443119125366211,
     'rmse': 9.066284,
     'val_loss': 8.045858383178711,
     'val_rmse': 11.971369}],
   [(0.4, 0.6000000000000001),
    {'loss': 7.147062463760376,
     'rmse': 10.180936,
     'val_loss': 63.91410827636719,
     'val_rmse': 106.37285}],
   [(0.6000000000000001, 0.8),
    {'loss': 8.451807041168212,
     'rmse': 16.859343,
     'val_loss': 7.305148124694824,
     'val_rmse': 10.13723}]]],
 [{'epochs': 25, 'layer_count': 5, 'node_number': 80, 'dropout': 0.8},
  [[(0.2, 0.4),
    {'loss': 5.365896091461182,
     'rmse': 7.8295307,
     'val_loss': 11.1212739944458,
     'val_rmse': 15.942946}],
   [(0.4, 0.6000000000000001),
    {'loss': 6.023767557144165,
     'rmse': 8.712482,
     'val_loss': 65.02237701416016,
     'val_rmse': 108.591484}],
   [(0.6000000000000001, 0.8),
    {'loss': 8.87853536605835,
     'rmse': 17.930183,
     'val_loss': 6.4038310050964355,


In [23]:
grid_search.evaluations.sort(key=lambda x:evaluator(x[1]))
grid_search.evaluations #Unievo

[[{'epochs': 25, 'layer_count': 7, 'node_number': 110, 'dropout': 0.8},
  [[(0.2, 0.4),
    {'loss': 0.09985854402184487,
     'rmse': 0.48959342,
     'val_loss': 50.34468078613281,
     'val_rmse': 218.95598}],
   [(0.4, 0.6000000000000001),
    {'loss': 6.384657497406006,
     'rmse': 45.841778,
     'val_loss': 361.4893493652344,
     'val_rmse': 428.8502}],
   [(0.6000000000000001, 0.8),
    {'loss': 105.14816802978515,
     'rmse': 204.56816,
     'val_loss': 163.1005401611328,
     'val_rmse': 197.17387}]]],
 [{'epochs': 25, 'layer_count': 5, 'node_number': 60, 'dropout': 0.8},
  [[(0.2, 0.4),
    {'loss': 0.09648234695196152,
     'rmse': 0.48970473,
     'val_loss': 50.34015655517578,
     'val_rmse': 218.95772}],
   [(0.4, 0.6000000000000001),
    {'loss': 39.36996440887451,
     'rmse': 170.5051,
     'val_loss': 455.2791442871094,
     'val_rmse': 526.3296}],
   [(0.6000000000000001, 0.8),
    {'loss': 213.36861389160157,
     'rmse': 468.7553,
     'val_loss': 326.29290771

## Evaluations on Evo

In [ ]:
feature_dict = {"epochs":[30], "layer_count":[5], "node_number":[80, 100, 120, 150], "dropout":[0.6, 0.7, 0.8, 0.9]}

grid_search.search(feature_dict, evo_data)

In [18]:
grid_search.evaluations #evo

[[{'epochs': 30, 'layer_count': 5, 'node_number': 120, 'dropout': 0.8},
  {'loss': 49.51356346130371,
   'rmse': 109.657265,
   'val_loss': 78.93724822998047,
   'val_rmse': 96.40157}],
 [{'epochs': 30, 'layer_count': 5, 'node_number': 100, 'dropout': 0.8},
  {'loss': 61.726402587890625,
   'rmse': 122.815506,
   'val_loss': 86.97623443603516,
   'val_rmse': 108.30454}],
 [{'epochs': 30, 'layer_count': 5, 'node_number': 80, 'dropout': 0.8},
  {'loss': 51.63334159851074,
   'rmse': 114.50193,
   'val_loss': 89.99913787841797,
   'val_rmse': 106.89356}],
 [{'epochs': 30, 'layer_count': 5, 'node_number': 100, 'dropout': 0.9},
  {'loss': 55.006822204589845,
   'rmse': 116.58102,
   'val_loss': 91.61908721923828,
   'val_rmse': 110.98308}],
 [{'epochs': 30, 'layer_count': 5, 'node_number': 80, 'dropout': 0.7},
  {'loss': 52.82768119812012,
   'rmse': 114.70442,
   'val_loss': 97.27867126464844,
   'val_rmse': 119.96011}],
 [{'epochs': 30, 'layer_count': 5, 'node_number': 80, 'dropout': 0.6}

In [16]:
grid_search.evaluations #unievo

[[{'epochs': 30, 'layer_count': 5, 'node_number': 60, 'dropout': 0.7},
  {'loss': 59.834226837158205,
   'rmse': 120.75018,
   'val_loss': 78.8970947265625,
   'val_rmse': 97.61479}],
 [{'epochs': 30, 'layer_count': 5, 'node_number': 120, 'dropout': 0.8},
  {'loss': 54.921137466430665,
   'rmse': 116.213264,
   'val_loss': 96.0332260131836,
   'val_rmse': 114.81347}],
 [{'epochs': 30, 'layer_count': 5, 'node_number': 80, 'dropout': 0.7},
  {'loss': 49.36703758239746,
   'rmse': 109.98419,
   'val_loss': 96.65272521972656,
   'val_rmse': 118.28333}],
 [{'epochs': 30, 'layer_count': 5, 'node_number': 100, 'dropout': 0.7},
  {'loss': 47.30355560302734,
   'rmse': 107.17045,
   'val_loss': 104.60826110839844,
   'val_rmse': 129.04442}],
 [{'epochs': 30, 'layer_count': 5, 'node_number': 100, 'dropout': 0.8},
  {'loss': 50.6439501953125,
   'rmse': 112.08298,
   'val_loss': 105.54985046386719,
   'val_rmse': 124.65917}],
 [{'epochs': 30, 'layer_count': 5, 'node_number': 80, 'dropout': 0.8},


In [17]:
grid_search.evaluations

[[{'epochs': 30, 'layer_count': 5, 'node_number': 100, 'dropout': 0.7},
  {'loss': 61.98115303039551,
   'rmse': 123.183075,
   'val_loss': 88.98165130615234,
   'val_rmse': 113.064804}],
 [{'epochs': 30, 'layer_count': 4, 'node_number': 60, 'dropout': 0.7},
  {'loss': 48.69632125854492,
   'rmse': 113.06194,
   'val_loss': 90.21286010742188,
   'val_rmse': 113.73388}],
 [{'epochs': 30, 'layer_count': 3, 'node_number': 60, 'dropout': 0.7},
  {'loss': 50.08780746459961,
   'rmse': 114.24129,
   'val_loss': 93.33883666992188,
   'val_rmse': 116.08402}],
 [{'epochs': 30, 'layer_count': 5, 'node_number': 60, 'dropout': 0.7},
  {'loss': 49.519894714355466,
   'rmse': 113.607605,
   'val_loss': 94.20116424560547,
   'val_rmse': 115.09389}],
 [{'epochs': 50, 'layer_count': 3, 'node_number': 100, 'dropout': 0.7},
  {'loss': 36.9069128036499,
   'rmse': 99.26817,
   'val_loss': 96.38170623779297,
   'val_rmse': 117.6753}],
 [{'epochs': 50, 'layer_count': 5, 'node_number': 80, 'dropout': 0.7},
 

In [17]:
feature_dict = {"epochs":[30, 50], "layer_count":[3, 4, 5], "node_number":[60, 80, 100], "dropout":[0.5, 0.7]}

grid_search.search(feature_dict, unievo_data, verbose=0)

[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]
[136, 181]



In [18]:
grid_search.evaluations

[[{'epochs': 30, 'layer_count': 5, 'node_number': 60, 'dropout': 0.5},
  {'loss': 49.16347648620606,
   'rmse': 111.650566,
   'val_loss': 91.99232482910156,
   'val_rmse': 114.38819}],
 [{'epochs': 30, 'layer_count': 5, 'node_number': 80, 'dropout': 0.7},
  {'loss': 60.79417556762695,
   'rmse': 121.694855,
   'val_loss': 92.54352569580078,
   'val_rmse': 113.08544}],
 [{'epochs': 30, 'layer_count': 3, 'node_number': 100, 'dropout': 0.7},
  {'loss': 49.68187171936035,
   'rmse': 106.38354,
   'val_loss': 96.49043273925781,
   'val_rmse': 119.60009}],
 [{'epochs': 30, 'layer_count': 4, 'node_number': 80, 'dropout': 0.7},
  {'loss': 48.58590560913086,
   'rmse': 108.54301,
   'val_loss': 98.93069458007812,
   'val_rmse': 121.50648}],
 [{'epochs': 50, 'layer_count': 3, 'node_number': 60, 'dropout': 0.7},
  {'loss': 41.57910179138184,
   'rmse': 103.59932,
   'val_loss': 99.25018310546875,
   'val_rmse': 122.40756}],
 [{'epochs': 30, 'layer_count': 5, 'node_number': 80, 'dropout': 0.5},
 

In [18]:
grid_search.evaluations

[[{'epochs': 50, 'layer_count': 4, 'node_number': 80, 'dropout': 0.7},
  {'RMSE': 162.35465398209146,
   'MAE': 98.66833072296089,
   'R2': 0.12723739996712005}],
 [{'epochs': 50, 'layer_count': 5, 'node_number': 80, 'dropout': 0.7},
  {'RMSE': 167.96424522934947,
   'MAE': 111.04613152448682,
   'R2': 0.0717189666230026}],
 [{'epochs': 30, 'layer_count': 4, 'node_number': 80, 'dropout': 0.7},
  {'RMSE': 169.5624757862915,
   'MAE': 114.56481393091921,
   'R2': 0.0519735506769354}],
 [{'epochs': 30, 'layer_count': 4, 'node_number': 80, 'dropout': 0.5},
  {'RMSE': 178.52453876247571,
   'MAE': 114.95373132738514,
   'R2': -0.05765273791179789}],
 [{'epochs': 50, 'layer_count': 4, 'node_number': 80, 'dropout': 0.5},
  {'RMSE': 169.96668573362294,
   'MAE': 115.84041734709257,
   'R2': 0.04534592232713058}],
 [{'epochs': 50, 'layer_count': 5, 'node_number': 80, 'dropout': 0.5},
  {'RMSE': 170.70319108707923,
   'MAE': 116.51601840102155,
   'R2': 0.04077082728880005}],
 [{'epochs': 30, 'l

In [16]:
grid_search.evaluations

[[{'epochs': 30, 'layer_count': 3, 'node_number': 80, 'dropout': 0.5},
  {'RMSE': 156.89216176717633,
   'MAE': 90.44141034889911,
   'R2': 0.18229860225635006}],
 [{'epochs': 50, 'layer_count': 3, 'node_number': 80, 'dropout': 0.7},
  {'RMSE': 160.75687936313233,
   'MAE': 93.47899978402732,
   'R2': 0.14363741376468986}],
 [{'epochs': 30, 'layer_count': 3, 'node_number': 80, 'dropout': 0.3},
  {'RMSE': 161.24735397338654,
   'MAE': 95.41294302629387,
   'R2': 0.1369881539715284}],
 [{'epochs': 30, 'layer_count': 3, 'node_number': 50, 'dropout': 0.5},
  {'RMSE': 160.3776318433857,
   'MAE': 96.25941219191621,
   'R2': 0.1456387522665716}],
 [{'epochs': 30, 'layer_count': 2, 'node_number': 50, 'dropout': 0.3},
  {'RMSE': 161.92682414714625,
   'MAE': 98.33593439796694,
   'R2': 0.13022077235481136}],
 [{'epochs': 50, 'layer_count': 2, 'node_number': 50, 'dropout': 0.7},
  {'RMSE': 164.11180620426185,
   'MAE': 98.4271030322365,
   'R2': 0.10759922701138487}],
 [{'epochs': 50, 'layer_co

## Evaluations on Modo

In [ ]:
grid_search.

In [16]:
feature_dict = {"epochs":[30], "layer_count":[5, 7], "node_number":[100, 150], "dropout":[0.7, 0.8]}

grid_search.search(feature_dict, modo_data)

Train for 50 steps, validate for 50 steps
Epoch 1/20
50/50 [==============================] - 20s 403ms/step - loss: 27.7333 - rmse: 39.0025 - val_loss: 31.9094 - val_rmse: 39.5464
Epoch 2/20
50/50 [==============================] - 13s 253ms/step - loss: 19.9862 - rmse: 31.1699 - val_loss: 33.2806 - val_rmse: 40.8856
Epoch 3/20
50/50 [==============================] - 13s 251ms/step - loss: 16.3877 - rmse: 28.6879 - val_loss: 17.0891 - val_rmse: 23.2675
Epoch 4/20
50/50 [==============================] - 11s 224ms/step - loss: 13.5137 - rmse: 26.3941 - val_loss: 19.9522 - val_rmse: 26.1082
Epoch 5/20
50/50 [==============================] - 11s 214ms/step - loss: 11.9810 - rmse: 25.1765 - val_loss: 14.9616 - val_rmse: 20.7518
Epoch 6/20
50/50 [==============================] - 11s 224ms/step - loss: 10.7354 - rmse: 24.0703 - val_loss: 16.8574 - val_rmse: 22.9561
Epoch 7/20
50/50 [==============================] - 11s 220ms/step - loss: 10.0270 - rmse: 23.5015 - val_loss: 16.3506 - val

KeyboardInterrupt: 

In [19]:
results = grid_search.run_lstm(modo_data, 24, 12, (0.6, 0.8), node_number=100,
                 epochs=30, dropout=0.7, layer_count=5)

Train for 50 steps, validate for 50 steps
Epoch 1/30
50/50 [==============================] - 195s 4s/step - loss: 384.0321 - rmse: 509.8246 - val_loss: 40.4084 - val_rmse: 59.7598
Epoch 2/30
50/50 [==============================] - 160s 3s/step - loss: 32.9628 - rmse: 46.4794 - val_loss: 39.6881 - val_rmse: 46.9302
Epoch 3/30
50/50 [==============================] - 86s 2s/step - loss: 23.8318 - rmse: 34.3204 - val_loss: 23.4886 - val_rmse: 29.9218
Epoch 4/30
50/50 [==============================] - 109s 2s/step - loss: 28.1330 - rmse: 39.8971 - val_loss: 24.6535 - val_rmse: 32.1018
Epoch 5/30
50/50 [==============================] - 110s 2s/step - loss: 26.5411 - rmse: 37.6602 - val_loss: 35.4291 - val_rmse: 43.4910
Epoch 6/30
50/50 [==============================] - 101s 2s/step - loss: 21.2352 - rmse: 32.6402 - val_loss: 17.4413 - val_rmse: 23.1176
Epoch 7/30
50/50 [==============================] - 144s 3s/step - loss: 16.8921 - rmse: 28.9768 - val_loss: 26.4167 - val_rmse: 33.449

In [ ]:
results2 = grid_search.run_lstm(modo_data, 24, 12, (0.6, 0.8), node_number=150,
                 epochs=30, dropout=0.8, layer_count=7)

Train for 50 steps, validate for 50 steps
Epoch 1/30
50/50 [==============================] - 268s 5s/step - loss: 2207.0080 - rmse: 2882.4858 - val_loss: 1218.7455 - val_rmse: 1505.8566
Epoch 2/30
50/50 [==============================] - 204s 4s/step - loss: 1006.0016 - rmse: 1240.2717 - val_loss: 1660.3737 - val_rmse: 2070.1179
Epoch 3/30
50/50 [==============================] - 177s 4s/step - loss: 1410.2239 - rmse: 1757.6869 - val_loss: 2112.9944 - val_rmse: 2639.4121
Epoch 4/30
50/50 [==============================] - 112s 2s/step - loss: 1020.1891 - rmse: 1372.0468 - val_loss: 433.1337 - val_rmse: 598.3810
Epoch 5/30
50/50 [==============================] - 174s 3s/step - loss: 635.0993 - rmse: 813.4850 - val_loss: 568.2524 - val_rmse: 707.5743
Epoch 6/30
50/50 [==============================] - 185s 4s/step - loss: 1116.8075 - rmse: 1450.4524 - val_loss: 1941.1669 - val_rmse: 2215.6226
Epoch 7/30
50/50 [==============================] - 193s 4s/step - loss: 877.6634 - rmse: 1106